#Kaggle Titanic Dataset

The following is an implementation of titanic classification challenge solution using stacking. This model scored 80% accuracy in the competition which put it at top 10% on the leaderboard. In this notebook, we will explore feature enginnering methods and stacking methods

In [2]:
import pandas as pd
import shutil
import glob, os

from google.colab import drive
drive.mount('/content/gdrive')

#get prediction results from base models
saved_model_files = glob.glob('/content/gdrive/My Drive/Colab Notebooks/titanic/saved_model/*.csv')
stacking_model_files = glob.glob('/content/gdrive/My Drive/Colab Notebooks/titanic/stacking_model/*.csv')
saved_models = []
stacking_models = []

#save results in a list
for file in saved_model_files:
    saved_models.append(pd.read_csv(file))
for file in stacking_model_files:
    stacking_models.append(pd.read_csv(file))
train_set = stacking_models.pop(0)
test_set = saved_models.pop(0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#merge each data with PassengerId
for model in stacking_models:
    train_set = pd.merge(train_set,model,on='PassengerId')
for model in saved_models:
    test_set = pd.merge(test_set,model,on='PassengerId')

In [0]:
#Redefine columns
train_set.columns = ['PassengerId','1','2','3','4','5']
test_set.columns = ['PassengerId','1','2','3','4','5']
#get labels
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/titanic/data/train.csv').pop('Survived')
#drop Id for training
features = train_set.drop(['PassengerId'],axis=1)

In [5]:
#use XGBoost model for training
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

#increase number of parameters for better result
params = {
        'n_estimators': [500,700,1000,1500,2000,3000],
        'learning_rate': [0.1,0.01,0.02,0.001,0.005,0.0001],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb_model = XGBClassifier(objective='binary:logistic')
skf = StratifiedKFold(n_splits=7)
skf.get_n_splits()

random_search = RandomizedSearchCV(xgb_model,
                                   param_distributions=params,
                                   n_iter=5, #try larger value of iterations for better performance
                                   scoring='accuracy',
                                   cv=skf.split(features,labels),
                                   verbose=3, 
                                   random_state=1001)
random_search.fit(features,labels)

Fitting 7 folds for each of 5 candidates, totalling 35 fits
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8359375, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8515625, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The tr

[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8984375, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8740157480314961, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8818897637795275, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8661417322834646, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.8, n_estimators=500, min_child_weight=10, max_depth=3, learning_rate=0.1, gamma=2, colsample_bytree=0.6, score=0.8968253968253969, total=   0.1s
[CV] subsample=0.8, n_estimators=500, min_child_weight=1, max_depth=4, learning_rate=0.0001, gamma=2, colsample_bytree=1.0 


...

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

[CV]  subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0, score=0.8740157480314961, total=   0.1s
[CV] subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0, score=0.8818897637795275, total=   0.1s
[CV] subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0 


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

[CV]  subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0, score=0.8661417322834646, total=   0.1s
[CV] subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=0.6, n_estimators=500, min_child_weight=10, max_depth=5, learning_rate=0.02, gamma=0.5, colsample_bytree=1.0, score=0.8968253968253969, total=   0.1s


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7f075e20ddb0>,
          error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'n_estimators': [500, 700, 1000, 1500, 2000, 3000], 'learning_rate': [0.1, 0.01, 0.02, 0.001, 0.005, 0.0001], 'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=

In [6]:
#print result
print(random_search.cv_results_)
print(random_search.best_estimator_.score(features,labels))
print(random_search.best_score_)

{'mean_fit_time': array([0.08863875, 0.16042137, 0.75685038, 0.34259135, 0.10828069]), 'std_fit_time': array([0.00166211, 0.00179101, 0.0042349 , 0.00209849, 0.00309793]), 'mean_score_time': array([0.00205786, 0.00262237, 0.00728072, 0.00299784, 0.00217114]), 'std_score_time': array([0.00031456, 0.00072793, 0.00027937, 0.00023809, 0.00016573]), 'param_subsample': masked_array(data=[0.8, 0.8, 0.8, 1.0, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[500, 500, 3000, 1500, 500],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[10, 1, 1, 1, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 4, 5, 4, 5],
             mask=[False, False, False, False, False],
       fill_value='?',
          

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


87% accuracy seems really good for a basic stacking model! Now, let's use this model for submission.

In [7]:
#use best results for submission
index = test_set.pop('PassengerId')
submit_predictions = random_search.best_estimator_.predict(test_set)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
#submit results
submit = pd.DataFrame(data={'PassengerId':index,'Survived':submit_predictions})
submit.to_csv('/content/gdrive/My Drive/Colab Notebooks/titanic/submit.csv',index=False)